# Text Data Cleaning

In [194]:
import nltk;
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string 
import json
from nltk.sentiment import SentimentIntensityAnalyzer
import json
import pandas as pd
import os, json
import glob
from nltk.corpus import wordnet as wn

### a) Read in the Json files

In [195]:
#path_to_json = '/Users/liumingqian/anly-501-project-liumingqian0511/data/00-raw-data/twitter_data/'
def readFiles(path):
    json_pattern = os.path.join(path,'*.json')
    file_list = glob.glob(json_pattern)
    df = pd.read_json(file_list[0]) # an empty list to store the data frames
    #print(df.head())
    for file in file_list[1:]:
        data = pd.read_json(file) # read data frame from json file
        #print(data.head())
        df = pd.concat([df,data],axis=0, ignore_index=True) # append the data frame to the list

    #df = pd.concat(dfs,sort=False) # concatenate all the data frames in the list.
    return df
df = readFiles('/Users/liumingqian/anly-501-project-liumingqian0511/data/00-raw-data/insurance_co')
df.drop_duplicates(subset = 'text', inplace = True)

In [196]:
df1 = pd.read_json('/Users/liumingqian/anly-501-project-liumingqian0511/data/00-raw-data/insurance_co/Aetna.json')
df2 = pd.read_json('/Users/liumingqian/anly-501-project-liumingqian0511/data/00-raw-data/insurance_co/Anthem.json')
df3 = pd.read_json('/Users/liumingqian/anly-501-project-liumingqian0511/data/00-raw-data/insurance_co/Cigna.json')
df4 = pd.read_json('/Users/liumingqian/anly-501-project-liumingqian0511/data/00-raw-data/insurance_co/Humana.json')
df5 = pd.read_json('/Users/liumingqian/anly-501-project-liumingqian0511/data/00-raw-data/insurance_co/UnitedHealth.json')


### b) Assign Labels Manually and Merge Data

In [197]:
df1['label'] = 'Aetna'
df2['label'] = 'Anthem'
df3['label'] = 'Cigna'
df4['label'] = 'Humana'
df5['label'] = 'UnitedHealth'


In [198]:
df = pd.concat([df1,df2,df3,df4,df5],ignore_index=True)
df.drop_duplicates(subset='text',inplace = True)
df = df[df['lang'] == 'en']
df.shape

(269, 7)

### c) Apply Sentiment Analysis

In [199]:
def getSentiments(df):
    sia = SentimentIntensityAnalyzer()
    tweet_str = ""
    tweetscore = []
    for tweet in df['text']:
        tweet_str = tweet_str + " " + tweet
        score = sia.polarity_scores(tweet_str)
        tweetscore.append(score)
    return tweetscore

sentiment = getSentiments(df)

In [200]:
score = pd.DataFrame.from_dict(sentiment)
score = score.assign(ID = list(range(1,len(score)+1)))
score.head(10)

,neg,neu,pos,compound,ID
0,0.000,1.000,0.000,0.0000,1
1,0.000,1.000,0.000,0.0000,2
2,0.000,1.000,0.000,0.0000,3
3,0.085,0.878,0.037,-0.7278,4
4,0.080,0.895,0.025,-0.8485,5
5,0.071,0.907,0.022,-0.8485,6
6,0.080,0.901,0.019,-0.9137,7
7,0.067,0.882,0.051,-0.6116,8
8,0.052,0.844,0.104,0.9533,9
9,0.048,0.840,0.112,0.9692,10


### d) Add Sentiment Outcome to Data

In [201]:
df = df.assign(ID = list(range(1,len(df)+1)))
df = df.merge(score,how='inner')
df['sentiment'] =df[['neg','neu','pos']].idxmax(axis=1)
df['sent_binary'] = df[['neg','pos']].idxmax(axis = 1)

### e) Filter out Stopwords

In [202]:
def filterStopwords(df):
    for val, tweet in enumerate(df['text']):
        new_text=""
        for word in nltk.tokenize.word_tokenize(tweet):
            word = word.lower()
            if word not in nltk.corpus.stopwords.words('english') and word not in [".",",","!","?",":",";",")","(","'","$",'https','co','rt','@','de','la','the']:
                    new_text+=word+" "
            df['text'][val] = new_text

filterStopwords(df)

/var/folders/0y/czdjt9k17_9fgxmg3d286zjm0000gn/T/ipykernel_67085/605275336.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][val] = new_text


### f) Lemmatize Text Column

In [203]:
lemmatizer = WordNetLemmatizer()
def lemmatize(df):
    for i,tweet in enumerate(df['text']):
        new_text = ""
        for word in tweet.split(' '):
            if wn.synsets(word):
                new_text += lemmatizer.lemmatize(word,pos= wn.synsets(word)[0].pos()) + " "
            else:
                new_text += word + " "
        df['text'][i] = new_text

In [204]:
lemmatize(df)
df.head()

/var/folders/0y/czdjt9k17_9fgxmg3d286zjm0000gn/T/ipykernel_67085/4089900633.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = new_text


,id,edit_history_tweet_ids,text,created_at,lang,author_id,label,ID,neg,neu,pos,compound,sentiment,sent_binary
0,1579944177819070464,[1579944177819070465],loyolaramblers 𝗧𝗵𝗲 𝗥𝗮𝗺𝗯𝗹𝗲𝗿 𝗥𝗼𝘂𝗻𝗱𝘁𝗮𝗯𝗹𝗲 present ...,2022-10-11 21:17:00+00:00,en,1015398717392687104,Aetna,1,0.000,1.000,0.000,0.0000,neu,neg
1,1579940514522370048,[1579940514522370048],pom_pnw aetna give directory resident house ad...,2022-10-11 21:02:27+00:00,en,1325197299409182720,Aetna,2,0.000,1.000,0.000,0.0000,neu,neg
2,1579937294152519680,[1579937294152519680],peds_ortho aetna ’ doctor ’ office take copay ...,2022-10-11 20:49:39+00:00,en,1228846428929380352,Aetna,3,0.000,1.000,0.000,0.0000,neu,neg
3,1579934089695875072,[1579934089695875072],aetna twice deny necessary medication b/c what...,2022-10-11 20:36:55+00:00,en,1077754392433082368,Aetna,4,0.085,0.878,0.037,-0.7278,neu,neg
4,1579932311864238080,[1579932311864238080],peds_ortho aetna pay copay meet oop limit ’ pa...,2022-10-11 20:29:51+00:00,en,1228846428929380352,Aetna,5,0.080,0.895,0.025,-0.8485,neu,neg


### g) Tidy Dataframe

In [205]:
df['created_at'] = df['created_at'].apply(lambda x: x.date)
df.rename(columns={'created_at':'date','lang':'language'},inplace = True)
df.drop(columns = ['author_id','id','edit_history_tweet_ids','neg','pos','neu','compound'],inplace = True)
df.head()

,text,date,language,label,ID,sentiment,sent_binary
0,loyolaramblers 𝗧𝗵𝗲 𝗥𝗮𝗺𝗯𝗹𝗲𝗿 𝗥𝗼𝘂𝗻𝗱𝘁𝗮𝗯𝗹𝗲 present ...,2022-10-11,en,Aetna,1,neu,neg
1,pom_pnw aetna give directory resident house ad...,2022-10-11,en,Aetna,2,neu,neg
2,peds_ortho aetna ’ doctor ’ office take copay ...,2022-10-11,en,Aetna,3,neu,neg
3,aetna twice deny necessary medication b/c what...,2022-10-11,en,Aetna,4,neu,neg
4,peds_ortho aetna pay copay meet oop limit ’ pa...,2022-10-11,en,Aetna,5,neu,neg


### h) Vectorization

In [206]:
corpus_str = ""
corpus = []
df['text'].apply(lambda x: corpus.append(x))
corpus_str = corpus_str.join(corpus)

In [207]:
vectorizer=CountVectorizer()
Xs  =  vectorizer.fit_transform(corpus)
X = np.array(Xs.todense())
col_names=vectorizer.get_feature_names_out()
vec = pd.DataFrame(X,columns=col_names)
vec['label'] = df['label']
vec.to_csv('/Users/liumingqian/anly-501-project-liumingqian0511/data/01-modified-data/vec.csv')
vec.head()

,00,000,01,02,08,0bl5dzrcc2,0cfllxypoq,0skzqs8f8h,10,100,...,yoy,yuvo,z4wc2sk8ji,zero,zoryalondonsk,zwsyhywsgy,𝗥𝗮𝗺𝗯𝗹𝗲𝗿,𝗥𝗼𝘂𝗻𝗱𝘁𝗮𝗯𝗹𝗲,𝗧𝗵𝗲,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,Aetna
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Aetna
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Aetna
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Aetna
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Aetna


In [208]:
sum_words = Xs.sum(axis=0) 
words_freq = [[word, sum_words[0, idx]] for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq_df = pd.DataFrame(words_freq,columns=['word','Frequency'])
words_freq_df.head(10)

,word,Frequency
0,co,146
1,cigna,81
2,aetna,77
3,anthem,53
4,unitedhealth,49
5,health,40
6,get,29
7,group,29
8,amp,28
9,plan,27


In [209]:
words_freq_df.to_csv('/Users/liumingqian/anly-501-project-liumingqian0511/data/01-modified-data/word_frequency_text.csv')
df.to_csv('/Users/liumingqian/anly-501-project-liumingqian0511/data/01-modified-data/text.csv')